In [19]:
import pandas as pd
import plotly.express as px
df= pd.read_csv("C:/Data Science/data/2024-03-27T11-52_export.csv")

In [22]:
from pycountry_convert import country_alpha2_to_country_name, country_name_to_country_alpha3

def convert_alpha2_to_alpha3(alpha_2):
    # Check if alpha_2 is not a string (e.g., NaN or other types)
    if not isinstance(alpha_2, str):
        return None
    if alpha_2.lower() == 'unknown':  # Now safe to call .lower()
        return None  # Return None for 'unknown' country codes
    try:
        # First, convert Alpha-2 to country name, then country name to Alpha-3
        country_name = country_alpha2_to_country_name(alpha_2)
        alpha_3 = country_name_to_country_alpha3(country_name)
        return alpha_3
    except Exception as e:
        print(f"Error converting country code {alpha_2}: {e}")
        return None  # Return None in case of conversion failure

# Apply the conversion function on the 'Country' column of your DataFrame
df['Country_alpha3'] = df['Country'].apply(convert_alpha2_to_alpha3)




In [23]:
df['roberta_neg'] = pd.to_numeric(df['roberta_neg'], errors='coerce')
df['roberta_neu'] = pd.to_numeric(df['roberta_neu'], errors='coerce')
df['roberta_pos'] = pd.to_numeric(df['roberta_pos'], errors='coerce')

In [26]:
df['avg_sentiment'] = df.apply(lambda x: (-1 * x['roberta_neg']) + (0 * x['roberta_neu']) + (1 * x['roberta_pos']), axis=1)


In [30]:
fig = px.choropleth(country_sentiment,
                    locations="Country_alpha3",
                    color="avg_sentiment",
                    hover_name="Country_alpha3",
                    locationmode='ISO-3',
                    color_continuous_scale=px.colors.diverging.RdYlGn,
                    title="Average Comment Sentiment by Country")

# Update layout for a black background
fig.update_layout(
    paper_bgcolor='black',  # Sets the background color of the paper to black
    plot_bgcolor='black',  # Sets the background color of the plot to black
    geo=dict(
        landcolor='black',  # Optionally, change the land color for more contrast
        showocean=True,       # Show ocean
        oceancolor='darkblue' # Optionally, change the ocean color
    ),
    title=dict(
        x=0.5,                # Center the title
        xanchor='center',     # Use the center of the title for positioning
        font=dict(
            size=24,          # Adjust the title font size
            color='white'     # Ensure the title is visible against the background
        )
    ),
    coloraxis_colorbar=dict(
        title='Sentiment',   # Title of the colorbar
        tickfont=dict(color='white'),  # Ensure colorbar ticks are visible
        titlefont=dict(color='white')  # Ensure colorbar title is visible
    )
)

# Update geographies' and annotations' text color for visibility
fig.update_geos(
    showcountries=True,     # Show country borders
    countrycolor="black"    # Ensure country borders are visible
)
fig.update_traces(marker_line_width=0.5, marker_line_color='white')  # For markers' borders
fig.update_annotations(font=dict(color='white'))  # For annotations like titles

fig.show()

